<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Treinamento_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports e definições de funções

In [144]:
# !pip install tensorflow
# !pip install keras

In [145]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [146]:
import pandas as pd
import numpy as np
import json
import string, re
import itertools
import nltk
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.callbacks import EarlyStopping
py.init_notebook_mode(connected=True)
%matplotlib inline
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import math
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
import matplotlib
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix

In [147]:
def tokenize(tweets_list):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(tweets_list)
    return tokenizer.texts_to_sequences(tweets_list), tokenizer

In [148]:
def pad(text_tokenized, length=None):
    return pad_sequences(text_tokenized, maxlen=length, padding='post')

In [149]:
def preprocess(tweets_list, max_text_length):
   
    preprocess_tweets_list, tweets_list_tokenizer = tokenize(tweets_list)

    preprocess_tweets_list = pad(preprocess_tweets_list, length=max_text_length)

    return preprocess_tweets_list, tweets_list_tokenizer

In [150]:
# CONSTANTES DOS RESULTADOS
REDE = 'NaiveBayes'

BASE_NAME = 'BASE 0'
# BASE_NAME = 'BASE 1'
# BASE_NAME = 'BASE 2'

# Caminho arquivo de saída
PATH_ARQ_SAIDA = "/content/drive/MyDrive/TCC/resultados/resultados.csv"

#  rede base otimizador acuracia val_loss learning_rate dropout batch_size roc_curve train_resume confusion_matrix
resultados = pd.read_csv(PATH_ARQ_SAIDA, index_col=0)
resultados

,rede,base,otimizador,acuracia,val_loss,learning_rate,dropout,batch_size,roc_curve,train_resume,confusion_matrix,epochs,rank
0,Bi-LSTM,BASE 0,Nadam,0.864151,0.359641,0.0001,0.1,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3}, ""a...","{""00"": 855, ""01"": 73, ""10"": 179, ""11"": 748}",4,2.402818
1,Bi-LSTM,BASE 0,Nadam,0.876550,0.302028,0.0001,0.2,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3}, ""a...","{""00"": 803, ""01"": 125, ""10"": 104, ""11"": 823}",4,2.902218
2,Bi-LSTM,BASE 0,Nadam,0.864151,0.318839,0.0001,0.3,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3}, ""a...","{""00"": 802, ""01"": 126, ""10"": 126, ""11"": 801}",4,2.710301
3,Bi-LSTM,BASE 0,RMSProp,0.852291,0.325597,0.0001,0.1,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3}, ""a...","{""00"": 800, ""01"": 128, ""10"": 146, ""11"": 781}",4,2.617622
4,Bi-LSTM,BASE 0,RMSProp,0.852291,0.327882,0.0001,0.2,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3}, ""a...","{""00"": 838, ""01"": 90, ""10"": 184, ""11"": 743}",4,2.599382
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,ShallowNet,BASE 2,RMSProp,0.509202,175.864090,0.0001,0.0,64,"{""false_positive_rate"": [0.0, 0.30061349693251...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 323, ""01"": 166, ""10"": 314, ""11"": 175}",5,0.002895
356,ShallowNet,BASE 2,SGD,0.500000,82.666985,0.0001,0.0,64,"{""false_positive_rate"": [0.0, 0.0, 0.006134969...","{""epoch"": {""0"": 0, ""1"": 1}, ""accuracy"": {""0"": ...","{""00"": 486, ""01"": 3, ""10"": 486, ""11"": 3}",2,0.006048
357,ShallowNet,BASE 2,Nadam,0.496933,135.945312,0.0010,0.0,64,"{""false_positive_rate"": [0.0, 0.25766871165644...","{""epoch"": {""0"": 0, ""1"": 1}, ""accuracy"": {""0"": ...","{""00"": 329, ""01"": 160, ""10"": 332, ""11"": 157}",2,0.003655
358,ShallowNet,BASE 2,RMSProp,0.516360,121.295631,0.0010,0.0,64,"{""false_positive_rate"": [0.0, 0.31901840490797...","{""epoch"": {""0"": 0, ""1"": 1}, ""accuracy"": {""0"": ...","{""00"": 290, ""01"": 199, ""10"": 274, ""11"": 215}",2,0.004257


## Lendo a Base

In [151]:
if BASE_NAME == 'BASE 0':
  # PATH_BASE0_NAO_PROCESSADA  -Base0 Bruta Não processada
  PATH_BASE = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base0_classificada_balanceada.csv"

if BASE_NAME == 'BASE 1':
  # PATH_BASE1_JUNTO_COM_DA_LEILA_BALANCEADA - Base1 Com StopWords
  PATH_BASE = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv"

if BASE_NAME == 'BASE 2':
  # PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA - Base2 Sem StopWords
  PATH_BASE = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv"

In [152]:
# # Base0 Bruta Não processada
# PATH_BASE_1_CLASSIFICADA_BALANCEADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1_classificada_balanceada.csv"
# #Base0 Bruta Não processada
# PATH_BASE_2_CLASSIFICADA_BALANCEADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_classificada_balanceada.csv"
# #MATRIZ_CBOW_300_BASE_1 = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base1JuntaHateENotHateDaLeila_balanceada_CBOW300.CSV", delimiter=',')
# #MATRIZ_CBOW_300_BASE_2_LEILA = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_maior4_menor25_limpissima_balanceada_CBOW300.CSV", delimiter=',')
# #MATRIZ_CBOW_300_BASE_2 = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_classificada_balanceada_CBOW300.csv", delimiter=',') #saida

In [153]:
tweets = pd.read_csv(PATH_BASE, index_col=0)
tweets

,text,label
0,RT @ThomasM71708305: @allantercalivre A Bahia ...,1
1,@allantercalivre @gen_heleno @CNNBrasil O Gene...,1
2,@allantercalivre @gen_heleno @CNNBrasil Alan t...,1
3,@allantercalivre @OlavoOpressor @opropriolavo ...,1
4,RT @ThomasM71708305: @allantercalivre A Bahia ...,1
...,...,...
9269,simplesmente amoooo https://t.co/I12jVsUdvb,0
9270,@Mary_Greicy Abençoado Ano Novo sua linda e q...,0
9271,"@MarcusConcolato leva eles com a gente, eles s...",0
9272,se tu NUNCA fumou maconha cole isso no seu mur...,0


In [154]:
# Pega a quantidade de palavras únicas dos tweets de toda a base
_, text_tokenizer = tokenize(tweets['text'])
text_vocab_size = len(text_tokenizer.word_index)
print("Vocabulary size:", text_vocab_size)

Vocabulary size: 24839


# Prepara os dados de train, test e validation da base


In [155]:
# Base0 NÃO Processada
PATH_BASE0_NAO_PROCESSADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base0_classificada_balanceada.csv"

# Base1 Com StopWords
PATH_BASE1_JUNTO_COM_DA_LEILA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv"

# Base2 Sem StopWords
PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv"

NUM_BASES = 3

In [156]:
# Leitura das bases
tweets_base0 = pd.read_csv(PATH_BASE0_NAO_PROCESSADA, index_col=0)
print(tweets_base0.shape)
tweets_base1 = pd.read_csv(PATH_BASE1_JUNTO_COM_DA_LEILA_BALANCEADA, index_col=0)
print(tweets_base1.shape)
tweets_base2 = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA, index_col=0)
print(tweets_base2.shape)

bases = [tweets_base0, tweets_base1, tweets_base2]

(9274, 2)
(16036, 2)
(4886, 2)


In [157]:
# Gera as quantidades das divisões de cada base (train 60%, test 20% e validation 20%)
list_test_size = []
list_train_size = [] # Só para verificação, não é usado posteriormente
list_val_size = []
input_datas = []
output_labels = []
for base in bases:
  val_size = base.shape[0]/5 # 20%
  val_size = int(math.ceil(val_size))

  test_size = base.shape[0]/5 # 20%
  test_size = int(math.ceil(test_size))

  train_size = base.shape[0] - (test_size+val_size)
  list_test_size.append(test_size)
  list_train_size.append(train_size)
  list_val_size.append(val_size)

  input_data, _ = base['text'], None
  input_datas.append(input_data)
  output_labels.append(base['label'])

print('list_train_size: '+str(list_train_size))
print('list_test_size: '+str(list_test_size))
print('list_val_size: '+str(list_val_size))

list_train_size: [5564, 9620, 2930]
list_test_size: [1855, 3208, 978]
list_val_size: [1855, 3208, 978]


In [158]:
# Divide cada base em 3 fatias (train, test e val). Para cada fatia temos os dados X e as labels Y
list_x_train = []
list_y_train = []
list_x_test = []
list_y_test = []
list_x_val = []
list_y_val = []
for i in range(NUM_BASES):
  x_main, x_test, y_main, y_test = train_test_split(input_datas[i], output_labels[i], test_size=list_test_size[i], stratify=output_labels[i], random_state=42)

  x_train, x_val, y_train, y_val = train_test_split(x_main, y_main, test_size=list_val_size[i], stratify=y_main, random_state=42)

  list_x_train.append(x_train)
  list_y_train.append(y_train)
  list_x_test.append(x_test)
  list_y_test.append(y_test)
  list_x_val.append(x_val)
  list_y_val.append(y_val)


# Execução modelo

In [159]:
from sklearn.naive_bayes import MultinomialNB # or any other NB model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [160]:
def train_model(classifier, input, output, input_test, output_test, input_val, output_val):
    # Train
    classifier.fit(input, output)
   
    # Val
    predictions = classifier.predict(input_val)
    
    AUC = accuracy_score(output_val, predictions)
    
    false_positive_rate, true_positive_rate, _ = roc_curve(output_val, predictions)     

    conf_mat = confusion_matrix(output_val, predictions)    
    
    # Test
    accuracy = classifier.score(input_test, output_test)  

     
    # print("----------------------------------------------------")
    # print(accuracy)
    # print(false_positive_rate)
    # print(true_positive_rate)
    # print(AUC)
    # print(conf_mat)
    # print("----------------------------------------------------")
    

    return accuracy, false_positive_rate, true_positive_rate, AUC, conf_mat

In [161]:
accuracy = []
for i in range(NUM_BASES):
  # create a count vectorizer object 
  count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
  count_vect.fit(bases[i]['text'])

  # transform the training and validation data using count vectorizer object
  train_x =  count_vect.transform(list_x_train[i])
  test_x =  count_vect.transform(list_x_test[i])
  val_x =  count_vect.transform(list_x_val[i])

  encoder = preprocessing.LabelEncoder() # !!!
  train_y = encoder.fit_transform(list_y_train[i])
  test_y = encoder.fit_transform(list_y_test[i])
  val_y = encoder.fit_transform(list_y_val[i])

  # Naive Bayes on Count Vectors
  accuracy, false_positive_rate, true_positive_rate, AUC, conf_mat = train_model(naive_bayes.MultinomialNB(), train_x, train_y, test_x, test_y, val_x, val_y )

  # Parâmetros
  NOME_REDE = "MultinomialNB"
  
  NOME_BASE = ''
  if i == 0:
    NOME_BASE = 'BASE 0'
  if i == 1:
    NOME_BASE = 'BASE 1'
  if i == 2:
    NOME_BASE ='BASE 2'

  NOME_OTIMIZADOR = 'NA'
  ACURACIA = accuracy
  LOSS = 'NA' 
  LEARNING_RATE = 0
  DROPOUT_RATE = 0
  BATCH = 0 
  ROC_CURVE = json.dumps({
      "false_positive_rate": list(false_positive_rate),
      "true_positive_rate": list(true_positive_rate),
      "score": AUC})
  
  TRAIN_RESUME = {}

  confusion = conf_mat
  CONFUSION_MATRIX = json.dumps({
      "00": int(confusion[0][0]),
      "01": int(confusion[0][1]),
      "10": int(confusion[1][0]),
      "11": int(confusion[1][1])
  })

  EPOCHS = 1
  RANK = 0

  # Salvando resultado do modelo
  registro_resultado = {resultados.columns[0]: NOME_REDE, 
                        resultados.columns[1]: NOME_BASE, 
                        resultados.columns[2]: NOME_OTIMIZADOR,
                        resultados.columns[3]: ACURACIA, 
                        resultados.columns[4]: LOSS,
                        resultados.columns[5]: LEARNING_RATE,
                        resultados.columns[6]: DROPOUT_RATE,
                        resultados.columns[7]: BATCH,
                        resultados.columns[8]: ROC_CURVE,
                        resultados.columns[9]: TRAIN_RESUME,
                        resultados.columns[10]: CONFUSION_MATRIX,
                        resultados.columns[11]: EPOCHS,
                        resultados.columns[12]: RANK,
                      }

  resultados = resultados.append(registro_resultado, ignore_index=True)


In [163]:
resultados.head()

,rede,base,otimizador,acuracia,val_loss,learning_rate,dropout,batch_size,roc_curve,train_resume,confusion_matrix,epochs,rank
0,Bi-LSTM,BASE 0,Nadam,0.864151,0.359641,0.0001,0.1,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3}, ""a...","{""00"": 855, ""01"": 73, ""10"": 179, ""11"": 748}",4,2.402818
1,Bi-LSTM,BASE 0,Nadam,0.876550,0.302028,0.0001,0.2,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3}, ""a...","{""00"": 803, ""01"": 125, ""10"": 104, ""11"": 823}",4,2.902218
2,Bi-LSTM,BASE 0,Nadam,0.864151,0.318839,0.0001,0.3,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3}, ""a...","{""00"": 802, ""01"": 126, ""10"": 126, ""11"": 801}",4,2.710301
3,Bi-LSTM,BASE 0,RMSProp,0.852291,0.325597,0.0001,0.1,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3}, ""a...","{""00"": 800, ""01"": 128, ""10"": 146, ""11"": 781}",4,2.617622
4,Bi-LSTM,BASE 0,RMSProp,0.852291,0.327882,0.0001,0.2,32,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3}, ""a...","{""00"": 838, ""01"": 90, ""10"": 184, ""11"": 743}",4,2.599382


In [164]:
resultados.tail(36)

,rede,base,otimizador,acuracia,val_loss,learning_rate,dropout,batch_size,roc_curve,train_resume,confusion_matrix,epochs,rank
327,ShallowNet,BASE 0,Nadam,0.536927,111.835114,0.0010,0.0,32,"{""false_positive_rate"": [0.0, 0.47788565264293...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 388, ""01"": 540, ""10"": 319, ""11"": 608}",5,0.004801
328,ShallowNet,BASE 0,RMSProp,0.599461,98.940826,0.0010,0.0,32,"{""false_positive_rate"": [0.0, 0.28263214670981...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 612, ""01"": 316, ""10"": 427, ""11"": 500}",5,0.006059
329,ShallowNet,BASE 0,SGD,0.500270,2.827455,0.0010,0.0,32,"{""false_positive_rate"": [0.0, 0.00755124056094...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2}, ""accuracy""...","{""00"": 23, ""01"": 905, ""10"": 22, ""11"": 905}",3,0.176933
330,ShallowNet,BASE 0,Nadam,0.537466,396.028351,0.0001,0.0,64,"{""false_positive_rate"": [0.0, 0.19848975188781...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2}, ""accuracy""...","{""00"": 711, ""01"": 217, ""10"": 641, ""11"": 286}",3,0.001357
331,ShallowNet,BASE 0,RMSProp,0.539084,219.354919,0.0001,0.0,64,"{""false_positive_rate"": [0.0, 0.30312837108953...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2}, ""accuracy""...","{""00"": 618, ""01"": 310, ""10"": 545, ""11"": 382}",3,0.002458
332,ShallowNet,BASE 0,SGD,0.514825,23.337458,0.0001,0.0,64,"{""false_positive_rate"": [0.0, 0.09816612729234...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2}, ""accuracy""...","{""00"": 700, ""01"": 228, ""10"": 672, ""11"": 255}",3,0.022060
333,ShallowNet,BASE 0,Nadam,0.551482,143.630539,0.0010,0.0,64,"{""false_positive_rate"": [0.0, 0.32146709816612...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2}, ""accuracy""...","{""00"": 580, ""01"": 348, ""10"": 484, ""11"": 443}",3,0.003840
334,ShallowNet,BASE 0,RMSProp,0.552561,117.841263,0.0010,0.0,64,"{""false_positive_rate"": [0.0, 0.20819848975188...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2}, ""accuracy""...","{""00"": 693, ""01"": 235, ""10"": 595, ""11"": 332}",3,0.004689
335,ShallowNet,BASE 0,SGD,0.497574,6.910354,0.0010,0.0,64,"{""false_positive_rate"": [0.0, 0.01510248112189...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2}, ""accuracy""...","{""00"": 21, ""01"": 907, ""10"": 25, ""11"": 902}",3,0.072004
336,ShallowNet,BASE 1,Nadam,0.581047,62.266346,0.0001,0.0,32,"{""false_positive_rate"": [0.0, 0.22007481296758...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 1107, ""01"": 497, ""10"": 847, ""11"": 757}",10,0.009332


In [165]:
# Caso queira deletar uma linha use o código abaixo
# Use a propriedade label para especificar o índice da linha
#resultados = resultados.drop(labels=1, axis=0)
#resultados = resultados.reset_index(drop=True)
#resultados

In [166]:
# Salva arquivo de saída
resultados.to_csv(PATH_ARQ_SAIDA)

In [167]:
# # word level tf-idf
# tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features = max_text_length)
# X = tfidf_vect.fit_transform(text_column)

# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, output_label, test_size=0.3, random_state=42)

# # Naive Bayes on Word Level TF IDF Vectors
# accuracy.append(train_model(naive_bayes.MultinomialNB(), train_x, train_y, valid_x))
# print ("NB, WordLevel TF-IDF: ", accuracy)

In [168]:
# # ngram level tf-idf
# tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,2), max_features=max_text_length)
# X = tfidf_vect_ngram.fit_transform(text_column)

# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, output_label, test_size=0.3, random_state=42)

# # Naive Bayes on Ngram Level TF IDF Vectors
# accuracy.append(train_model(naive_bayes.MultinomialNB(), train_x, train_y, valid_x))
# print ("NB, N-Gram(1,2) Vectors: ", accuracy)

In [169]:
# # ngram level tf-idf
# tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=max_text_length)
# X = tfidf_vect_ngram.fit_transform(text_column)

# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, output_label, test_size=0.3, random_state=42)

# # Naive Bayes on Ngram Level TF IDF Vectors
# accuracy.append(train_model(naive_bayes.MultinomialNB(), train_x, train_y, valid_x))
# print ("NB, N-Gram(2,3) Vectors: ", accuracy)

In [170]:
# # characters level tf-idf
# tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=max_text_length)
# X = tfidf_vect_ngram_chars.fit_transform(text_column) 

# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, output_label, test_size=0.3, random_state=42)

# # Naive Bayes on Character Level TF IDF Vectors
# accuracy.append(train_model(naive_bayes.MultinomialNB(), train_x, train_y, valid_x))
# print ("NB, N-Gram(2,3) CharLevel Vectors: ", accuracy)

In [171]:
# # characters level tf-idf
# tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(1,120), max_features=max_text_length)
# X = tfidf_vect_ngram_chars.fit_transform(text_column) 

# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, output_label, test_size=0.3, random_state=2)

# # Naive Bayes on Character Level TF IDF Vectors
# accuracy.append(train_model(naive_bayes.MultinomialNB(), train_x, train_y, valid_x))
# print ("NB, N-Gram(2,3) CharLevel Vectors: ", accuracy)